In [48]:
num_unique_users = data_clean['user_encoded'].nunique()
num_unique_businesses = data_clean['business_encoded'].nunique()
print(f"총 고유 사용자 수: {num_unique_users:,}명")
print(f"총 고유 비즈니스 수: {num_unique_businesses:,}개")
total_elements = interaction_matrix.shape[0] * interaction_matrix.shape[1]
non_zero_elements = np.count_nonzero(interaction_matrix)
sparsity = (1 - (non_zero_elements / total_elements)) * 100
print(f"상호작용 행렬 희소성: {sparsity:.2f}%")

총 고유 사용자 수: 27,807명
총 고유 비즈니스 수: 6,831개
상호작용 행렬 희소성: 99.79%


In [52]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np
from collections import defaultdict
from tqdm import tqdm

# --- Surprise Library Imports ---
from surprise import Dataset, Reader, SVD
from surprise.model_selection import LeaveOneOut # Use surprise's LOO for proper internal handling

# --- 1. 데이터 로드 및 전처리 ---
# 데이터 로드
data = pd.read_json('review_business_5up_5aspect_3sentiment_vectorized_clean.json', lines=True)

# 필요한 컬럼만 추출
data_clean = data[['user_id', 'business_id', 'stars']].copy() # .copy()를 사용하여 SettingWithCopyWarning 방지

# 사용자와 비즈니스 아이디를 숫자로 인코딩 (Surprise에 전달하기 전에 필요)
user_encoder = LabelEncoder()
business_encoder = LabelEncoder()

data_clean.loc[:, 'user_encoded'] = user_encoder.fit_transform(data_clean['user_id'])
data_clean.loc[:, 'business_encoded'] = business_encoder.fit_transform(data_clean['business_id'])

# 중복되는 사용자-비즈니스 조합에 대해 평균 평점 계산 (Surprise는 단일 평점 기대)
data_clean = data_clean.groupby(['user_encoded', 'business_encoded'], as_index=False)['stars'].mean()

print(f"총 고유 사용자 수: {data_clean['user_encoded'].nunique():,}명")
print(f"총 고유 비즈니스 수: {data_clean['business_encoded'].nunique():,}개")

# --- 2. Surprise 라이브러리용 데이터 로드 및 LOO 분할 ---
# Reader는 평점 스케일을 지정합니다. (예: 1점 ~ 5점)
reader = Reader(rating_scale=(1, 5))
# data_clean DataFrame에서 user, item, rating 컬럼을 지정하여 Surprise Dataset으로 로드
data_surprise = Dataset.load_from_df(data_clean[['user_encoded', 'business_encoded', 'stars']], reader)

# Leave-One-Out 분할 (Surprise 내부 기능 활용)
# n_splits=1은 각 사용자당 하나의 아이템만 테스트 세트로 분리합니다.
loo_kf = LeaveOneOut(n_splits=1, random_state=42)

# next(loo_kf.split(data_surprise))를 통해 단일 훈련/테스트 세트 얻기
# trainset은 surprise.Trainset 객체, testset_surprise는 (uid, iid, r_ui) 튜플 리스트
trainset, testset_surprise = next(loo_kf.split(data_surprise))

print(f"학습 데이터셋 크기 (Surprise): {trainset.n_ratings:,}개")
print(f"테스트 데이터셋 크기 (Surprise): {len(testset_surprise):,}개")

# --- 3. SVD 모델 학습 (Surprise SVD) ---
# n_factors는 잠재 변수 개수
# n_epochs는 학습 반복 횟수
# lr_all은 학습률, reg_all은 정규화 파라미터
algo = SVD(n_factors=500, random_state=42, n_epochs=20, lr_all=0.005, reg_all=0.02)
algo.fit(trainset)

# --- 4. Precision@K 계산 함수 (Surprise에 맞춰 수정) ---
def precision_at_k_surprise(predictions, k=5):
    """
    Surprise의 SVD 모델을 사용하여 LOO Precision@K를 계산합니다.
    predictions: algo.test()에서 반환된 예측 리스트 (uid, iid, true_r, est, _) 튜플 형태
    """
    # 테스트 세트에 있는 각 사용자의 실제 아이템을 매핑 (외부 ID 사용)
    user_to_actual_item = {}
    for uid, iid, _, _, _ in predictions: # uid: 사용자 외부 ID, iid: 아이템 외부 ID
        user_to_actual_item[uid] = iid

    # 모든 사용자에 대해 추천 목록 생성 (훈련 세트에서 보지 않은 아이템만 대상으로)
    all_recommendations = {}
    
    # trainset에 있는 모든 사용자 내부 ID를 순회합니다.
    for user_inner_id in tqdm(trainset.all_users(), desc="Generating recommendations"):
        uid = trainset.to_raw_uid(user_inner_id) # 내부 ID를 원본 (외부) 사용자 ID로 변환

        # 이 사용자가 훈련 세트에서 이미 본 아이템의 내부 ID 목록
        seen_items_inner_ids = [item_inner_id for (item_inner_id, _) in trainset.ur[user_inner_id]]
        # 내부 ID를 외부 ID로 변환
        seen_items_outer_ids = [trainset.to_raw_iid(item_inner_id) for item_inner_id in seen_items_inner_ids]

        # 이 사용자가 훈련 세트에서 보지 않은 아이템들의 외부 ID 목록을 생성
        unseen_items_outer_ids = []
        # Surprise의 모든 아이템 내부 ID를 순회
        for item_inner_id in trainset.all_items():
            iid = trainset.to_raw_iid(item_inner_id) # 내부 ID를 원본 (외부) 아이템 ID로 변환
            if iid not in seen_items_outer_ids: # 본 적 없는 아이템만 추가
                unseen_items_outer_ids.append(iid)

        # 보지 않은 아이템들에 대한 예측 수행
        # predict()는 사용자 ID와 아이템 ID가 "원본(raw) 형태"일 것을 기대합니다.
        # 따라서 to_raw_uid/iid로 변환된 값들을 사용해야 합니다.
        # 또한, Surprise는 trainset에 없는 (즉, 예측할 필요 없는) 조합에 대해서도 predict를 호출할 수 있으므로,
        # 실제로 predict 가능한 아이템만 예측하도록 필터링하는 것이 안전합니다 (내부적으로 처리되기도 함).
        unseen_predictions = [algo.predict(uid, iid) for iid in unseen_items_outer_ids]

        # 예측 점수를 기준으로 내림차순 정렬하여 상위 k개 아이템 선택
        unseen_predictions.sort(key=lambda x: x.est, reverse=True)
        top_k_recs = [pred.iid for pred in unseen_predictions[:k]] # pred.iid는 예측된 아이템의 외부 ID
        all_recommendations[uid] = top_k_recs

    # Precision 계산
    precisions = []
    # 테스트 세트에 있는 각 사용자-실제 아이템 쌍에 대해
    for uid, actual_iid in user_to_actual_item.items():
        if uid in all_recommendations: # 해당 사용자에 대한 추천이 생성되었다면
            if actual_iid in all_recommendations[uid]: # 실제 아이템이 추천 목록에 있다면 Hit!
                precisions.append(1)
            else: # Hit이 아니라면 Miss
                precisions.append(0)
        # else: 해당 사용자가 trainset에 없어 추천이 생성되지 않은 경우는 무시 (LOO에서는 대부분 있음)
            
    return np.mean(precisions) if precisions else 0.0 # 평균 Precision 반환

# 모델의 예측 결과 (테스트 세트에 있는 각 사용자-숨겨진 아이템 쌍에 대한 예측)
test_predictions = algo.test(testset_surprise)

# Precision@5 계산 및 출력
precision_surprise = precision_at_k_surprise(test_predictions, k=5)
print(f"Precision@5 (Surprise SVD): {precision_surprise:.4f}")

총 고유 사용자 수: 27,807명
총 고유 비즈니스 수: 6,831개
학습 데이터셋 크기 (Surprise): 401,146개
테스트 데이터셋 크기 (Surprise): 27,807개


Generating recommendations: 100%|██████████| 27807/27807 [14:31<00:00, 31.92it/s]

Precision@5 (Surprise SVD): 0.0031


In [53]:
# Precision@5 계산 및 출력
precision_surprise = precision_at_k_surprise(test_predictions, k=100)
print(f"Precision@5 (Surprise SVD): {precision_surprise:.4f}")

Generating recommendations: 100%|██████████| 27807/27807 [14:34<00:00, 31.79it/s]

Precision@5 (Surprise SVD): 0.0351


In [58]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np

# --- Surprise Library Imports ---
from surprise import Dataset, Reader, SVD
from surprise.model_selection import KFold # RMSE 평가에 KFold 사용
from surprise import accuracy # RMSE를 계산하기 위해 필요

# --- 1. 데이터 로드 및 전처리 (이전과 동일) ---
data = pd.read_json('review_business_5up_5aspect_3sentiment_vectorized_clean.json', lines=True)

# 필요한 컬럼 추출
data_clean = data[['user_id', 'business_id', 'stars']].copy() # sentiment_vector는 일단 제외

# 사용자와 비즈니스 아이디를 숫자로 인코딩
user_encoder = LabelEncoder()
business_encoder = LabelEncoder()

data_clean.loc[:, 'user_encoded'] = user_encoder.fit_transform(data_clean['user_id'])
data_clean.loc[:, 'business_encoded'] = business_encoder.fit_transform(data_clean['business_id'])

# 사용자-비즈니스 쌍에 대해 평균 평점 계산 (동일한 user_id, business_id 쌍에 여러 리뷰가 있다면)
data_aggregated = data_clean.groupby(['user_encoded', 'business_encoded'], as_index=False)['stars'].mean()

print(f"총 고유 사용자 수: {data_aggregated['user_encoded'].nunique():,}명")
print(f"총 고유 비즈니스 수: {data_aggregated['business_encoded'].nunique():,}개")
print(f"총 집계된 평점 데이터 수: {len(data_aggregated):,}개")

# --- 2. Surprise 라이브러리용 데이터 로드 ---
reader = Reader(rating_scale=(1, 5)) # 별점 스케일 지정 (예: 1~5)
data_surprise = Dataset.load_from_df(data_aggregated[['user_encoded', 'business_encoded', 'stars']], reader)

# --- 3. K-Fold 교차 검증 준비 ---
kf = KFold(n_splits=5, random_state=42, shuffle=True) # 5-Fold 교차 검증

# --- 4. SVD 모델 학습 및 RMSE 평가 ---
# SVD 모델 파라미터 설정 (이전과 동일하게 유지)
svd_algo = SVD(n_factors=500, random_state=42, n_epochs=20, lr_all=0.005, reg_all=0.02, verbose=False)

# RMSE를 저장할 리스트
rmses = []

print("\n--- 기존 SVD 모델 K-Fold 교차 검증 시작 (ABSA 미반영) ---")
for i, (trainset, testset) in enumerate(kf.split(data_surprise)):
    print(f"Fold {i+1}/{kf.n_splits} - 학습 중...")
    
    # SVD 모델 학습
    svd_algo.fit(trainset)
    
    # 테스트 세트에 대한 예측 생성
    predictions = svd_algo.test(testset)
    
    # RMSE 계산
    rmse = accuracy.rmse(predictions, verbose=False)
    rmses.append(rmse)
    print(f"Fold {i+1} 완료. RMSE: {rmse:.4f}")

print("\n--- 기존 SVD 모델 RMSE 결과 (ABSA 미반영) ---")
print(f"각 Fold의 RMSE: {rmses}")
print(f"평균 RMSE: {np.mean(rmses):.4f}")

총 고유 사용자 수: 27,807명
총 고유 비즈니스 수: 6,831개
총 집계된 평점 데이터 수: 428,953개

--- 기존 SVD 모델 K-Fold 교차 검증 시작 (ABSA 미반영) ---
Fold 1/5 - 학습 중...
Fold 1 완료. RMSE: 1.0613
Fold 2/5 - 학습 중...
Fold 2 완료. RMSE: 1.0627
Fold 3/5 - 학습 중...
Fold 3 완료. RMSE: 1.0615
Fold 4/5 - 학습 중...
Fold 4 완료. RMSE: 1.0620
Fold 5/5 - 학습 중...
Fold 5 완료. RMSE: 1.0659

--- 기존 SVD 모델 RMSE 결과 (ABSA 미반영) ---
각 Fold의 RMSE: [1.0612557336852626, 1.062658806645337, 1.0615232983255034, 1.0619605971398314, 1.0658725392919286]
평균 RMSE: 1.0627


In [57]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

# --- Surprise Library Imports ---
from surprise import Dataset, Reader, SVD
from surprise.model_selection import KFold # RMSE 평가에는 KFold가 더 일반적
from surprise import accuracy

# --- 1. 데이터 로드 및 전처리 ---
data = pd.read_json('review_business_5up_5aspect_3sentiment_vectorized_clean.json', lines=True)

# 필요한 컬럼 추출
data_clean = data[['user_id', 'business_id', 'stars', 'sentiment_vector']].copy()

# 사용자와 비즈니스 아이디를 숫자로 인코딩
user_encoder = LabelEncoder()
business_encoder = LabelEncoder()

data_clean.loc[:, 'user_encoded'] = user_encoder.fit_transform(data_clean['user_id'])
data_clean.loc[:, 'business_encoded'] = business_encoder.fit_transform(data_clean['business_id'])

# sentiment_vector 컬럼을 각 요소별로 분리하여 새로운 컬럼 생성
sentiment_vector_df = pd.DataFrame(data_clean['sentiment_vector'].tolist(),
                                   index=data_clean.index,
                                   columns=[f'sentiment_vector_{i}' for i in range(len(data_clean['sentiment_vector'].iloc[0]))])

data_clean = pd.concat([data_clean.drop('sentiment_vector', axis=1), sentiment_vector_df], axis=1)

# ABSA 피처 컬럼 이름 목록 생성
absa_feature_cols = [col for col in data_clean.columns if 'sentiment_vector_' in col]

# 사용자-비즈니스-평점/ABSA 피처 집계 (평균)
group_cols = ['user_encoded', 'business_encoded']
agg_funcs = {'stars': 'mean'}
for col in absa_feature_cols:
    agg_funcs[col] = 'mean'

data_aggregated = data_clean.groupby(group_cols, as_index=False).agg(agg_funcs)

print(f"총 고유 사용자 수: {data_aggregated['user_encoded'].nunique():,}명")
print(f"총 고유 비즈니스 수: {data_aggregated['business_encoded'].nunique():,}개")
print(f"총 집계된 평점/ABSA 데이터 수: {len(data_aggregated):,}개")

# --- 2. Surprise SVD 모델 학습 및 예측 준비 ---
reader = Reader(rating_scale=(1, 5))
data_surprise = Dataset.load_from_df(data_aggregated[['user_encoded', 'business_encoded', 'stars']], reader)
kf = KFold(n_splits=5, random_state=42, shuffle=True) # 5-Fold 교차 검증

# --- 3. ABSA 기반 콘텐츠 모델을 위한 데이터 준비 ---

# 각 비즈니스별 ABSA 피처 평균 계산 (아이템 콘텐츠 벡터)
business_absa_features_avg = data_aggregated.groupby('business_encoded')[absa_feature_cols].mean()

# 코사인 유사도 계산을 위해 DataFrame을 numpy 배열로 변환
absa_matrix = business_absa_features_avg.values
# 비즈니스 ID 인덱스 매핑 (나중에 유사도 행렬에서 ID를 찾기 위해)
business_id_to_idx = {business_id: idx for idx, business_id in enumerate(business_absa_features_avg.index)}
idx_to_business_id = {idx: business_id for business_id, idx in business_id_to_idx.items()}

# 아이템-아이템 코사인 유사도 계산 (ABSA 피처 기반)
print("\n--- ABSA 기반 아이템-아이템 유사도 행렬 계산 중 (시간 소요) ---")
item_absa_similarity_matrix = cosine_similarity(absa_matrix)
print("ABSA 기반 아이템-아이템 유사도 행렬 계산 완료.")

# --- 4. 앙상블을 위한 교차 검증 및 예측 ---

svd_predictions_list = []
absa_predictions_list = []

print("\n--- 하이브리드 앙상블 모델 K-Fold 교차 검증 시작 ---")

for fold_idx, (trainset, testset) in enumerate(kf.split(data_surprise)):
    print(f"\nFold {fold_idx+1}/{kf.n_splits} - 학습 및 예측 중...")

    # --- 4-1. SVD 모델 학습 및 예측 ---
    svd_algo = SVD(n_factors=500, random_state=42, n_epochs=20, lr_all=0.005, reg_all=0.02, verbose=False)
    svd_algo.fit(trainset)
    svd_fold_predictions = svd_algo.test(testset)
    svd_predictions_list.extend(svd_fold_predictions)


    # --- 4-2. ABSA 기반 콘텐츠 모델 예측 ---
    absa_fold_predictions = []
    
    # 테스트 세트의 각 (사용자, 아이템) 쌍에 대해 예측 수행
    for uid_outer, iid_outer, true_r in tqdm(testset, desc=f"Fold {fold_idx+1} ABSA Prediction"):
        
        # 사용자가 훈련 세트에서 이미 평가한 아이템 정보 가져오기
        try:
            user_inner_id = trainset.to_inner_uid(uid_outer)
            # (item_inner_id, rating) 튜플 리스트
            user_ratings_in_train = trainset.ur[user_inner_id]
        except ValueError: # 테스트 유저가 훈련셋에 없는 경우 (드묾)
            absa_fold_predictions.append((uid_outer, iid_outer, true_r, np.mean(trainset.global_mean), {})) # 평균으로 대체
            continue

        # 사용자 u가 훈련 세트에서 평가한 비즈니스 ID 및 평점 목록
        rated_items_by_user = []
        for item_inner_id, rating in user_ratings_in_train:
            # 수정된 부분: to_outer_iid 대신 to_raw_iid 사용
            rated_items_by_user.append((trainset.to_raw_iid(item_inner_id), rating))

        # 타겟 아이템의 ABSA 기반 예측 평점 계산
        predicted_absa_rating = np.mean(trainset.global_mean) # 기본값은 전체 평균
        
        # 타겟 아이템이 ABSA 데이터에 있는지 확인
        if iid_outer not in business_id_to_idx:
            absa_fold_predictions.append((uid_outer, iid_outer, true_r, predicted_absa_rating, {}))
            continue # 다음 아이템으로 넘어감

        target_item_absa_idx = business_id_to_idx[iid_outer]
        
        weighted_sum = 0
        similarity_sum = 0
        
        # 사용자가 훈련 세트에서 평가했던 각 아이템에 대해
        for rated_item_outer_id, rated_rating in rated_items_by_user:
            if rated_item_outer_id in business_id_to_idx:
                rated_item_absa_idx = business_id_to_idx[rated_item_outer_id]
                
                # 타겟 아이템과 평가했던 아이템 간의 ABSA 유사도
                similarity = item_absa_similarity_matrix[target_item_absa_idx, rated_item_absa_idx]
                
                # 유사도가 0보다 크다면 가중치로 사용 (0이면 상관 없다고 가정)
                if similarity > 0:
                    weighted_sum += similarity * rated_rating
                    similarity_sum += similarity
        
        if similarity_sum > 0:
            predicted_absa_rating = weighted_sum / similarity_sum
        else:
            # 유사한 아이템을 찾지 못했다면, 해당 사용자의 평균 평점 또는 전체 평균으로 대체
            try:
                predicted_absa_rating = trainset.mean(user_inner_id) # 사용자 평균 평점
            except ValueError: # 사용자 평균도 없을 때 (매우 드묾)
                predicted_absa_rating = trainset.global_mean # 전체 평균
        
        absa_fold_predictions.append((uid_outer, iid_outer, true_r, predicted_absa_rating, {}))
    
    absa_predictions_list.extend(absa_fold_predictions)

# --- 5. 예측 결과 결합 및 RMSE 측정 ---

# SVD와 ABSA 예측 결과를 정렬하여 (user, item) 쌍이 일치하도록 합니다.
svd_preds_df = pd.DataFrame([(p.uid, p.iid, p.est, p.r_ui) for p in svd_predictions_list], columns=['uid', 'iid', 'svd_est', 'true_r'])
absa_preds_df = pd.DataFrame([(p[0], p[1], p[3]) for p in absa_predictions_list], columns=['uid', 'iid', 'absa_est'])

# 실제 평점은 SVD 예측 리스트에서 가져오는 것이 가장 정확합니다.
combined_preds_df = pd.merge(svd_preds_df, absa_preds_df, on=['uid', 'iid'], how='inner')

# 다양한 가중치 조합으로 RMSE 평가 (Grid Search)
best_rmse = float('inf')
best_w_svd = 0
best_w_absa = 0

# 가중치 탐색 범위
weights_svd = np.linspace(0.1, 0.9, 9) # 0.1부터 0.9까지 0.1 간격
weights_absa = 1 - weights_svd # 두 가중치의 합이 1이 되도록

print("\n--- 앙상블 가중치 조합 탐색 시작 ---")
results = []
for w_svd, w_absa in zip(weights_svd, weights_absa):
    combined_preds_df['ensemble_est'] = (w_svd * combined_preds_df['svd_est']) + \
                                        (w_absa * combined_preds_df['absa_est'])
    
    # 평점 스케일 범위(1~5)를 벗어나지 않도록 클리핑 (선택 사항이지만 권장)
    combined_preds_df['ensemble_est'] = np.clip(combined_preds_df['ensemble_est'], 1, 5)

    rmse = np.sqrt(np.mean((combined_preds_df['ensemble_est'] - combined_preds_df['true_r'])**2))
    
    results.append({'w_svd': w_svd, 'w_absa': w_absa, 'rmse': rmse})
    print(f"w_svd: {w_svd:.1f}, w_absa: {w_absa:.1f}, RMSE: {rmse:.4f}")

    if rmse < best_rmse:
        best_rmse = rmse
        best_w_svd = w_svd
        best_w_absa = w_absa

print("\n--- 앙상블 결과 ---")
print(f"최적의 RMSE: {best_rmse:.4f}")
print(f"최적의 가중치 (SVD: {best_w_svd:.1f}, ABSA: {best_w_absa:.1f})")

총 고유 사용자 수: 27,807명
총 고유 비즈니스 수: 6,831개
총 집계된 평점/ABSA 데이터 수: 428,953개

--- ABSA 기반 아이템-아이템 유사도 행렬 계산 중 (시간 소요) ---
ABSA 기반 아이템-아이템 유사도 행렬 계산 완료.

--- 하이브리드 앙상블 모델 K-Fold 교차 검증 시작 ---

Fold 1/5 - 학습 및 예측 중...


Fold 1 ABSA Prediction: 100%|██████████| 85791/85791 [00:03<00:00, 21568.93it/s]



Fold 2/5 - 학습 및 예측 중...


Fold 2 ABSA Prediction: 100%|██████████| 85791/85791 [00:04<00:00, 19220.77it/s]



Fold 3/5 - 학습 및 예측 중...


...


Fold 4/5 - 학습 및 예측 중...


Fold 4 ABSA Prediction: 100%|██████████| 85790/85790 [00:04<00:00, 19162.28it/s]



Fold 5/5 - 학습 및 예측 중...


Fold 5 ABSA Prediction: 100%|██████████| 85790/85790 [00:03<00:00, 22448.06it/s]



--- 앙상블 가중치 조합 탐색 시작 ---
w_svd: 0.1, w_absa: 0.9, RMSE: 1.1091
w_svd: 0.2, w_absa: 0.8, RMSE: 1.0927
w_svd: 0.3, w_absa: 0.7, RMSE: 1.0789
w_svd: 0.4, w_absa: 0.6, RMSE: 1.0678
w_svd: 0.5, w_absa: 0.5, RMSE: 1.0596
w_svd: 0.6, w_absa: 0.4, RMSE: 1.0543
w_svd: 0.7, w_absa: 0.3, RMSE: 1.0520
w_svd: 0.8, w_absa: 0.2, RMSE: 1.0526
w_svd: 0.9, w_absa: 0.1, RMSE: 1.0562

--- 앙상블 결과 ---
최적의 RMSE: 1.0520
최적의 가중치 (SVD: 0.7, ABSA: 0.3)


In [61]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np

# --- Surprise Library Imports ---
from surprise import Dataset, Reader, SVD
from surprise.model_selection import KFold # RMSE 평가에 KFold 사용
from surprise import accuracy # RMSE와 MAE 계산을 위해 필요

# --- 1. 데이터 로드 및 전처리 ---
data = pd.read_json('review_business_5up_5aspect_3sentiment_vectorized_clean.json', lines=True)

# 필요한 컬럼 추출
data_clean = data[['user_id', 'business_id', 'stars']].copy() # sentiment_vector는 일단 제외

# 사용자와 비즈니스 아이디를 숫자로 인코딩
user_encoder = LabelEncoder()
business_encoder = LabelEncoder()

data_clean.loc[:, 'user_encoded'] = user_encoder.fit_transform(data_clean['user_id'])
data_clean.loc[:, 'business_encoded'] = business_encoder.fit_transform(data_clean['business_id'])

# 사용자-비즈니스 쌍에 대해 평균 평점 계산
data_aggregated = data_clean.groupby(['user_encoded', 'business_encoded'], as_index=False)['stars'].mean()

print(f"총 고유 사용자 수: {data_aggregated['user_encoded'].nunique():,}명")
print(f"총 고유 비즈니스 수: {data_aggregated['business_encoded'].nunique():,}개")
print(f"총 집계된 평점 데이터 수: {len(data_aggregated):,}개")

# --- 2. Surprise 라이브러리용 데이터 로드 ---
reader = Reader(rating_scale=(1, 5)) # 별점 스케일 지정 (예: 1~5)
data_surprise = Dataset.load_from_df(data_aggregated[['user_encoded', 'business_encoded', 'stars']], reader)

# --- 3. K-Fold 교차 검증 준비 ---
kf = KFold(n_splits=5, random_state=42, shuffle=True) # 5-Fold 교차 검증

# --- 4. SVD 모델 학습 및 RMSE, MAE 평가 ---
# SVD 모델 파라미터 설정
svd_algo = SVD(n_factors=500, random_state=42, n_epochs=20, lr_all=0.005, reg_all=0.02, verbose=False)

# RMSE와 MAE를 저장할 리스트
rmses = []
maes = []

print("\n--- 기존 SVD 모델 K-Fold 교차 검증 시작 (ABSA 미반영) ---")
for i, (trainset, testset) in enumerate(kf.split(data_surprise)):
    print(f"Fold {i+1}/{kf.n_splits} - 학습 중...")
    
    # SVD 모델 학습
    svd_algo.fit(trainset)
    
    # 테스트 세트에 대한 예측 생성
    predictions = svd_algo.test(testset)
    
    # RMSE 계산
    rmse = accuracy.rmse(predictions, verbose=False)
    rmses.append(rmse)
    
    # MAE 계산
    mae = accuracy.mae(predictions, verbose=False)
    maes.append(mae)
    
    print(f"Fold {i+1} 완료. RMSE: {rmse:.4f}, MAE: {mae:.4f}")

print("\n--- 기존 SVD 모델 RMSE, MAE 결과 (ABSA 미반영) ---")
print(f"각 Fold의 RMSE: {rmses}")
print(f"평균 RMSE: {np.mean(rmses):.4f}")
print(f"각 Fold의 MAE: {maes}")
print(f"평균 MAE: {np.mean(maes):.4f}")


총 고유 사용자 수: 27,807명
총 고유 비즈니스 수: 6,831개
총 집계된 평점 데이터 수: 428,953개

--- 기존 SVD 모델 K-Fold 교차 검증 시작 (ABSA 미반영) ---
Fold 1/5 - 학습 중...
Fold 1 완료. RMSE: 1.0613, MAE: 0.8298
Fold 2/5 - 학습 중...
Fold 2 완료. RMSE: 1.0627, MAE: 0.8325
Fold 3/5 - 학습 중...
Fold 3 완료. RMSE: 1.0615, MAE: 0.8321
Fold 4/5 - 학습 중...
Fold 4 완료. RMSE: 1.0620, MAE: 0.8320
Fold 5/5 - 학습 중...
Fold 5 완료. RMSE: 1.0659, MAE: 0.8345

--- 기존 SVD 모델 RMSE, MAE 결과 (ABSA 미반영) ---
각 Fold의 RMSE: [1.0612557336852626, 1.062658806645337, 1.0615232983255034, 1.0619605971398314, 1.0658725392919286]
평균 RMSE: 1.0627
각 Fold의 MAE: [0.8297796587574818, 0.8325079182605186, 0.8320716937758054, 0.8319797545594015, 0.8345271118773612]
평균 MAE: 0.8322


In [60]:
# --- 5. 예측 결과 결합 및 RMSE, MAE 측정 ---

# SVD와 ABSA 예측 결과를 정렬하여 (user, item) 쌍이 일치하도록 합니다.
svd_preds_df = pd.DataFrame([(p.uid, p.iid, p.est, p.r_ui) for p in svd_predictions_list], columns=['uid', 'iid', 'svd_est', 'true_r'])
absa_preds_df = pd.DataFrame([(p[0], p[1], p[3]) for p in absa_predictions_list], columns=['uid', 'iid', 'absa_est'])

# 실제 평점은 SVD 예측 리스트에서 가져오는 것이 가장 정확합니다.
combined_preds_df = pd.merge(svd_preds_df, absa_preds_df, on=['uid', 'iid'], how='inner')

# 다양한 가중치 조합으로 RMSE, MAE 평가 (Grid Search)
best_rmse = float('inf')
best_mae = float('inf')  # MAE에 대한 변수 추가
best_w_svd = 0
best_w_absa = 0

# 가중치 탐색 범위
weights_svd = np.linspace(0.1, 0.9, 9) # 0.1부터 0.9까지 0.1 간격
weights_absa = 1 - weights_svd # 두 가중치의 합이 1이 되도록

print("\n--- 앙상블 가중치 조합 탐색 시작 ---")
results = []
for w_svd, w_absa in zip(weights_svd, weights_absa):
    combined_preds_df['ensemble_est'] = (w_svd * combined_preds_df['svd_est']) + \
                                        (w_absa * combined_preds_df['absa_est'])
    
    # 평점 스케일 범위(1~5)를 벗어나지 않도록 클리핑 (선택 사항이지만 권장)
    combined_preds_df['ensemble_est'] = np.clip(combined_preds_df['ensemble_est'], 1, 5)

    # RMSE 계산
    rmse = np.sqrt(np.mean((combined_preds_df['ensemble_est'] - combined_preds_df['true_r'])**2))

    # MAE 계산
    mae = np.mean(np.abs(combined_preds_df['ensemble_est'] - combined_preds_df['true_r']))

    results.append({'w_svd': w_svd, 'w_absa': w_absa, 'rmse': rmse, 'mae': mae})
    print(f"w_svd: {w_svd:.1f}, w_absa: {w_absa:.1f}, RMSE: {rmse:.4f}, MAE: {mae:.4f}")

    # 최적 RMSE, MAE 값 업데이트
    if rmse < best_rmse:
        best_rmse = rmse
        best_w_svd = w_svd
        best_w_absa = w_absa

    if mae < best_mae:
        best_mae = mae
        best_w_svd_mae = w_svd
        best_w_absa_mae = w_absa

print("\n--- 앙상블 결과 ---")
print(f"최적의 RMSE: {best_rmse:.4f}")
print(f"최적의 가중치 (SVD: {best_w_svd:.1f}, ABSA: {best_w_absa:.1f})")

# 최적 MAE 값도 출력
print(f"최적의 MAE: {best_mae:.4f}")
print(f"최적의 가중치 (SVD: {best_w_svd_mae:.1f}, ABSA: {best_w_absa_mae:.1f})")



--- 앙상블 가중치 조합 탐색 시작 ---
w_svd: 0.1, w_absa: 0.9, RMSE: 1.1091, MAE: 0.8576
w_svd: 0.2, w_absa: 0.8, RMSE: 1.0927, MAE: 0.8478
w_svd: 0.3, w_absa: 0.7, RMSE: 1.0789, MAE: 0.8397
w_svd: 0.4, w_absa: 0.6, RMSE: 1.0678, MAE: 0.8332
w_svd: 0.5, w_absa: 0.5, RMSE: 1.0596, MAE: 0.8285
w_svd: 0.6, w_absa: 0.4, RMSE: 1.0543, MAE: 0.8256
w_svd: 0.7, w_absa: 0.3, RMSE: 1.0520, MAE: 0.8246
w_svd: 0.8, w_absa: 0.2, RMSE: 1.0526, MAE: 0.8254
w_svd: 0.9, w_absa: 0.1, RMSE: 1.0562, MAE: 0.8280

--- 앙상블 결과 ---
최적의 RMSE: 1.0520
최적의 가중치 (SVD: 0.7, ABSA: 0.3)
최적의 MAE: 0.8246
최적의 가중치 (SVD: 0.7, ABSA: 0.3)


In [64]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

# --- Surprise Library Imports ---
from surprise import Dataset, Reader, SVD
from surprise.model_selection import KFold
from surprise import accuracy

# --- 1. 데이터 로드 및 전처리 ---
data = pd.read_json('review_business_5up_5aspect_3sentiment_vectorized_clean.json', lines=True)

# 필요한 컬럼 추출
data_clean = data[['user_id', 'business_id', 'stars', 'sentiment_vector']].copy()

# 사용자와 비즈니스 아이디를 숫자로 인코딩
user_encoder = LabelEncoder()
business_encoder = LabelEncoder()

data_clean.loc[:, 'user_encoded'] = user_encoder.fit_transform(data_clean['user_id'])
data_clean.loc[:, 'business_encoded'] = business_encoder.fit_transform(data_clean['business_id'])

# sentiment_vector 컬럼을 각 요소별로 분리
sentiment_vector_df = pd.DataFrame(data_clean['sentiment_vector'].tolist(),
                                   index=data_clean.index,
                                   columns=[f'sentiment_vector_{i}' for i in range(len(data_clean['sentiment_vector'].iloc[0]))])

data_clean = pd.concat([data_clean.drop('sentiment_vector', axis=1), sentiment_vector_df], axis=1)

# --- 새로운 ABSA 피처 엔지니어링: 각 측면별 순 감성 점수 추출 ---
# 가정: sentiment_vector는 3개씩 묶여 5개 측면의 (부정, 중립, 긍정) 감성 점수를 나타냄
new_absa_feature_cols = []
num_aspects = 5 # 5개 측면을 가정
for i in range(num_aspects):
    neg_col = f'sentiment_vector_{i*3}' # 부정 감성 인덱스
    pos_col = f'sentiment_vector_{i*3+2}' # 긍정 감성 인덱스
    net_sentiment_col_name = f'aspect_{i+1}_net_sentiment'
    data_clean[net_sentiment_col_name] = data_clean[pos_col] - data_clean[neg_col]
    new_absa_feature_cols.append(net_sentiment_col_name)

print(f"새로 생성된 ABSA 피처 컬럼: {new_absa_feature_cols}")

# 사용자-비즈니스-평점/ABSA 피처 집계 (평균)
group_cols = ['user_encoded', 'business_encoded']
agg_funcs = {'stars': 'mean'}
for col in new_absa_feature_cols:
    agg_funcs[col] = 'mean'

data_aggregated = data_clean.groupby(group_cols, as_index=False).agg(agg_funcs)

print(f"총 고유 사용자 수: {data_aggregated['user_encoded'].nunique():,}명")
print(f"총 고유 비즈니스 수: {data_aggregated['business_encoded'].nunique():,}개")
print(f"총 집계된 평점/ABSA 데이터 수: {len(data_aggregated):,}개")

# --- 2. Surprise SVD 모델 학습 및 예측 준비 ---
reader = Reader(rating_scale=(1, 5))
data_surprise = Dataset.load_from_df(data_aggregated[['user_encoded', 'business_encoded', 'stars']], reader)
kf = KFold(n_splits=5, random_state=42, shuffle=True)

# --- 3. ABSA 기반 콘텐츠 모델을 위한 데이터 준비 ---

# 각 비즈니스별 새로운 ABSA 피처 평균 계산 (아이템 콘텐츠 벡터)
business_absa_features_avg = data_aggregated.groupby('business_encoded')[new_absa_feature_cols].mean()

# 코사인 유사도 계산을 위해 DataFrame을 numpy 배열로 변환
absa_matrix = business_absa_features_avg.values
# 비즈니스 ID 인덱스 매핑
business_id_to_idx = {business_id: idx for idx, business_id in enumerate(business_absa_features_avg.index)}
idx_to_business_id = {idx: business_id for business_id, idx in business_id_to_idx.items()}

# 아이템-아이템 코사인 유사도 계산 (새로운 ABSA 피처 기반)
print("\n--- ABSA 기반 아이템-아이템 유사도 행렬 계산 중 (시간 소요) ---")
item_absa_similarity_matrix = cosine_similarity(absa_matrix)
print("ABSA 기반 아이템-아이템 유사도 행렬 계산 완료.")

# --- 4. 앙상블을 위한 교차 검증 및 예측 ---

svd_predictions_list = []
absa_predictions_list = []

print("\n--- 하이브리드 앙상블 모델 K-Fold 교차 검증 시작 (ABSA 피처 엔지니어링 반영) ---")

for fold_idx, (trainset, testset) in enumerate(kf.split(data_surprise)):
    print(f"\nFold {fold_idx+1}/{kf.n_splits} - 학습 및 예측 중...")

    # --- 4-1. SVD 모델 학습 및 예측 ---
    svd_algo = SVD(n_factors=500, random_state=42, n_epochs=20, lr_all=0.005, reg_all=0.02, verbose=False)
    svd_algo.fit(trainset)
    svd_fold_predictions = svd_algo.test(testset)
    svd_predictions_list.extend(svd_fold_predictions)


    # --- 4-2. ABSA 기반 콘텐츠 모델 예측 ---
    absa_fold_predictions = []
    
    for uid_outer, iid_outer, true_r in tqdm(testset, desc=f"Fold {fold_idx+1} ABSA Prediction"):
        
        try:
            user_inner_id = trainset.to_inner_uid(uid_outer)
            user_ratings_in_train = trainset.ur[user_inner_id] # (item_inner_id, rating) 튜플 리스트
            
            # 사용자 평균 평점 계산 (수정된 부분)
            # 해당 사용자가 훈련 세트에서 평가한 평점들이 존재할 때만 계산
            if user_ratings_in_train: 
                user_avg_rating = np.mean([r for _, r in user_ratings_in_train])
            else: # 평가한 아이템이 없다면 전체 평균
                user_avg_rating = trainset.global_mean

        except ValueError: # 테스트 유저가 훈련셋에 없는 경우
            # 이 경우 user_avg_rating은 trainset.global_mean으로 설정
            user_avg_rating = trainset.global_mean
            user_ratings_in_train = [] # 평가한 아이템 없음을 표시
            
        rated_items_by_user = []
        for item_inner_id, rating in user_ratings_in_train:
            rated_items_by_user.append((trainset.to_raw_iid(item_inner_id), rating))

        predicted_absa_rating = user_avg_rating # 기본값은 사용자 평균 평점 (없다면 전체 평균)
        
        if iid_outer not in business_id_to_idx:
            absa_fold_predictions.append((uid_outer, iid_outer, true_r, predicted_absa_rating, {}))
            continue

        target_item_absa_idx = business_id_to_idx[iid_outer]
        
        weighted_sum = 0
        similarity_sum = 0
        
        for rated_item_outer_id, rated_rating in rated_items_by_user:
            if rated_item_outer_id in business_id_to_idx:
                rated_item_absa_idx = business_id_to_idx[rated_item_outer_id]
                
                similarity = item_absa_similarity_matrix[target_item_absa_idx, rated_item_absa_idx]
                
                if similarity > 0:
                    weighted_sum += similarity * rated_rating
                    similarity_sum += similarity
        
        if similarity_sum > 0:
            predicted_absa_rating = weighted_sum / similarity_sum
        else:
            # 유사한 아이템을 찾지 못했다면, 이전에 계산한 사용자 평균 평점 사용
            predicted_absa_rating = user_avg_rating
        
        absa_fold_predictions.append((uid_outer, iid_outer, true_r, predicted_absa_rating, {}))
    
    absa_predictions_list.extend(absa_fold_predictions)

# --- 5. 예측 결과 결합 및 RMSE 측정 ---

svd_preds_df = pd.DataFrame([(p.uid, p.iid, p.est, p.r_ui) for p in svd_predictions_list], columns=['uid', 'iid', 'svd_est', 'true_r'])
absa_preds_df = pd.DataFrame([(p[0], p[1], p[3]) for p in absa_predictions_list], columns=['uid', 'iid', 'absa_est'])

combined_preds_df = pd.merge(svd_preds_df, absa_preds_df, on=['uid', 'iid'], how='inner')

best_rmse = float('inf')
best_mae = float('inf')  # MAE에 대한 변수 추가
best_w_svd = 0
best_w_absa = 0

weights_svd = np.linspace(0.1, 0.9, 9)
weights_absa = 1 - weights_svd

print("\n--- 앙상블 가중치 조합 탐색 시작 (ABSA 피처 엔지니어링 반영) ---")
results = []
for w_svd, w_absa in zip(weights_svd, weights_absa):
    combined_preds_df['ensemble_est'] = (w_svd * combined_preds_df['svd_est']) + \
                                        (w_absa * combined_preds_df['absa_est'])
    
    combined_preds_df['ensemble_est'] = np.clip(combined_preds_df['ensemble_est'], 1, 5)

    rmse = np.sqrt(np.mean((combined_preds_df['ensemble_est'] - combined_preds_df['true_r'])**2))
    
    results.append({'w_svd': w_svd, 'w_absa': w_absa, 'rmse': rmse})
    print(f"w_svd: {w_svd:.1f}, w_absa: {w_absa:.1f}, RMSE: {rmse:.4f}")

    if rmse < best_rmse:
        best_rmse = rmse
        best_w_svd = w_svd
        best_w_absa = w_absa

    if mae < best_mae:
        best_mae = mae
        best_w_svd_mae = w_svd
        best_w_absa_mae = w_absa


print("\n--- 앙상블 결과 (ABSA 피처 엔지니어링 반영) ---")
print(f"최적의 RMSE: {best_rmse:.4f}")
print(f"최적의 가중치 (SVD: {best_w_svd:.1f}, ABSA: {best_w_absa:.1f})")

# 최적 MAE 값도 출력
print(f"최적의 MAE: {best_mae:.4f}")
print(f"최적의 가중치 (SVD: {best_w_svd_mae:.1f}, ABSA: {best_w_absa_mae:.1f})")


새로 생성된 ABSA 피처 컬럼: ['aspect_1_net_sentiment', 'aspect_2_net_sentiment', 'aspect_3_net_sentiment', 'aspect_4_net_sentiment', 'aspect_5_net_sentiment']
총 고유 사용자 수: 27,807명
총 고유 비즈니스 수: 6,831개
총 집계된 평점/ABSA 데이터 수: 428,953개

--- ABSA 기반 아이템-아이템 유사도 행렬 계산 중 (시간 소요) ---
ABSA 기반 아이템-아이템 유사도 행렬 계산 완료.

--- 하이브리드 앙상블 모델 K-Fold 교차 검증 시작 (ABSA 피처 엔지니어링 반영) ---

Fold 1/5 - 학습 및 예측 중...


Fold 1 ABSA Prediction: 100%|██████████| 85791/85791 [00:03<00:00, 23884.49it/s]



Fold 2/5 - 학습 및 예측 중...


Fold 2 ABSA Prediction: 100%|██████████| 85791/85791 [00:03<00:00, 23872.01it/s]



Fold 3/5 - 학습 및 예측 중...


...


Fold 4/5 - 학습 및 예측 중...


Fold 4 ABSA Prediction: 100%|██████████| 85790/85790 [00:03<00:00, 24115.50it/s]



Fold 5/5 - 학습 및 예측 중...


Fold 5 ABSA Prediction: 100%|██████████| 85790/85790 [00:03<00:00, 24067.60it/s]



--- 앙상블 가중치 조합 탐색 시작 (ABSA 피처 엔지니어링 반영) ---
w_svd: 0.1, w_absa: 0.9, RMSE: 1.1057
w_svd: 0.2, w_absa: 0.8, RMSE: 1.0889
w_svd: 0.3, w_absa: 0.7, RMSE: 1.0749
w_svd: 0.4, w_absa: 0.6, RMSE: 1.0638
w_svd: 0.5, w_absa: 0.5, RMSE: 1.0558
w_svd: 0.6, w_absa: 0.4, RMSE: 1.0509
w_svd: 0.7, w_absa: 0.3, RMSE: 1.0491
w_svd: 0.8, w_absa: 0.2, RMSE: 1.0505
w_svd: 0.9, w_absa: 0.1, RMSE: 1.0550

--- 앙상블 결과 (ABSA 피처 엔지니어링 반영) ---
최적의 RMSE: 1.0491
최적의 가중치 (SVD: 0.7, ABSA: 0.3)
최적의 MAE: 0.8345
최적의 가중치 (SVD: 0.1, ABSA: 0.9)


In [65]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

# --- Surprise Library Imports ---
from surprise import Dataset, Reader, SVD
from surprise.model_selection import KFold
from surprise import accuracy

# --- 1. 데이터 로드 및 전처리 ---
data = pd.read_json('review_business_5up_5aspect_3sentiment_vectorized_clean.json', lines=True)

# 필요한 컬럼 추출
data_clean = data[['user_id', 'business_id', 'stars', 'sentiment_vector']].copy()

# 사용자와 비즈니스 아이디를 숫자로 인코딩
user_encoder = LabelEncoder()
business_encoder = LabelEncoder()

data_clean.loc[:, 'user_encoded'] = user_encoder.fit_transform(data_clean['user_id'])
data_clean.loc[:, 'business_encoded'] = business_encoder.fit_transform(data_clean['business_id'])

# sentiment_vector 컬럼을 각 요소별로 분리하여 새로운 컬럼 생성
# 이번에는 15개 모든 차원을 피처로 사용합니다.
sentiment_vector_df = pd.DataFrame(data_clean['sentiment_vector'].tolist(),
                                   index=data_clean.index,
                                   columns=[f'sentiment_vector_{i}' for i in range(len(data_clean['sentiment_vector'].iloc[0]))])

data_clean = pd.concat([data_clean.drop('sentiment_vector', axis=1), sentiment_vector_df], axis=1)

# ABSA 피처 컬럼 이름 목록 생성 (모든 15개 차원)
absa_feature_cols = [col for col in data_clean.columns if 'sentiment_vector_' in col]

print(f"사용할 ABSA 피처 컬럼: {absa_feature_cols}")
print(f"총 ABSA 피처 개수: {len(absa_feature_cols)}개")


# 사용자-비즈니스-평점/ABSA 피처 집계 (평균)
group_cols = ['user_encoded', 'business_encoded']
agg_funcs = {'stars': 'mean'}
for col in absa_feature_cols: # 모든 15개 피처를 집계에 포함
    agg_funcs[col] = 'mean'

data_aggregated = data_clean.groupby(group_cols, as_index=False).agg(agg_funcs)

print(f"총 고유 사용자 수: {data_aggregated['user_encoded'].nunique():,}명")
print(f"총 고유 비즈니스 수: {data_aggregated['business_encoded'].nunique():,}개")
print(f"총 집계된 평점/ABSA 데이터 수: {len(data_aggregated):,}개")

# --- 2. Surprise SVD 모델 학습 및 예측 준비 ---
reader = Reader(rating_scale=(1, 5))
data_surprise = Dataset.load_from_df(data_aggregated[['user_encoded', 'business_encoded', 'stars']], reader)
kf = KFold(n_splits=5, random_state=42, shuffle=True)

# --- 3. ABSA 기반 콘텐츠 모델을 위한 데이터 준비 ---

# 각 비즈니스별 15개 ABSA 피처 평균 계산 (아이템 콘텐츠 벡터)
business_absa_features_avg = data_aggregated.groupby('business_encoded')[absa_feature_cols].mean()

# 코사인 유사도 계산을 위해 DataFrame을 numpy 배열로 변환
absa_matrix = business_absa_features_avg.values
# 비즈니스 ID 인덱스 매핑
business_id_to_idx = {business_id: idx for idx, business_id in enumerate(business_absa_features_avg.index)}
idx_to_business_id = {idx: business_id for business_id, idx in business_id_to_idx.items()}

# 아이템-아이템 코사인 유사도 계산 (15개 ABSA 피처 기반)
print("\n--- ABSA 기반 아이템-아이템 유사도 행렬 계산 중 (시간 소요) ---")
item_absa_similarity_matrix = cosine_similarity(absa_matrix)
print("ABSA 기반 아이템-아이템 유사도 행렬 계산 완료.")

# --- 4. 앙상블을 위한 교차 검증 및 예측 ---

svd_predictions_list = []
absa_predictions_list = []

print("\n--- 하이브리드 앙상블 모델 K-Fold 교차 검증 시작 (15차원 ABSA 피처 반영) ---")

for fold_idx, (trainset, testset) in enumerate(kf.split(data_surprise)):
    print(f"\nFold {fold_idx+1}/{kf.n_splits} - 학습 및 예측 중...")

    # --- 4-1. SVD 모델 학습 및 예측 ---
    svd_algo = SVD(n_factors=500, random_state=42, n_epochs=20, lr_all=0.005, reg_all=0.02, verbose=False)
    svd_algo.fit(trainset)
    svd_fold_predictions = svd_algo.test(testset)
    svd_predictions_list.extend(svd_fold_predictions)


    # --- 4-2. ABSA 기반 콘텐츠 모델 예측 ---
    absa_fold_predictions = []
    
    for uid_outer, iid_outer, true_r in tqdm(testset, desc=f"Fold {fold_idx+1} ABSA Prediction"):
        
        try:
            user_inner_id = trainset.to_inner_uid(uid_outer)
            user_ratings_in_train = trainset.ur[user_inner_id]
            
            if user_ratings_in_train: 
                user_avg_rating = np.mean([r for _, r in user_ratings_in_train])
            else:
                user_avg_rating = trainset.global_mean

        except ValueError:
            user_avg_rating = trainset.global_mean
            user_ratings_in_train = []
            
        rated_items_by_user = []
        for item_inner_id, rating in user_ratings_in_train:
            rated_items_by_user.append((trainset.to_raw_iid(item_inner_id), rating))

        predicted_absa_rating = user_avg_rating
        
        if iid_outer not in business_id_to_idx:
            absa_fold_predictions.append((uid_outer, iid_outer, true_r, predicted_absa_rating, {}))
            continue

        target_item_absa_idx = business_id_to_idx[iid_outer]
        
        weighted_sum = 0
        similarity_sum = 0
        
        for rated_item_outer_id, rated_rating in rated_items_by_user:
            if rated_item_outer_id in business_id_to_idx:
                rated_item_absa_idx = business_id_to_idx[rated_item_outer_id]
                
                similarity = item_absa_similarity_matrix[target_item_absa_idx, rated_item_absa_idx]
                
                if similarity > 0:
                    weighted_sum += similarity * rated_rating
                    similarity_sum += similarity
        
        if similarity_sum > 0:
            predicted_absa_rating = weighted_sum / similarity_sum
        else:
            predicted_absa_rating = user_avg_rating
        
        absa_fold_predictions.append((uid_outer, iid_outer, true_r, predicted_absa_rating, {}))
    
    absa_predictions_list.extend(absa_fold_predictions)

# --- 5. 예측 결과 결합 및 RMSE 측정 ---

svd_preds_df = pd.DataFrame([(p.uid, p.iid, p.est, p.r_ui) for p in svd_predictions_list], columns=['uid', 'iid', 'svd_est', 'true_r'])
absa_preds_df = pd.DataFrame([(p[0], p[1], p[3]) for p in absa_predictions_list], columns=['uid', 'iid', 'absa_est'])

combined_preds_df = pd.merge(svd_preds_df, absa_preds_df, on=['uid', 'iid'], how='inner')

best_rmse = float('inf')
best_w_svd = 0
best_w_absa = 0
best_mae = float('inf') # MAE도 함께 추적
best_w_svd_mae = 0
best_w_absa_mae = 0

weights_svd = np.linspace(0.1, 0.9, 9)
weights_absa = 1 - weights_svd

print("\n--- 앙상블 가중치 조합 탐색 시작 (15차원 ABSA 피처 반영) ---")
results = []
for w_svd, w_absa in zip(weights_svd, weights_absa):
    combined_preds_df['ensemble_est'] = (w_svd * combined_preds_df['svd_est']) + \
                                        (w_absa * combined_preds_df['absa_est'])
    
    combined_preds_df['ensemble_est'] = np.clip(combined_preds_df['ensemble_est'], 1, 5)

    rmse = np.sqrt(np.mean((combined_preds_df['ensemble_est'] - combined_preds_df['true_r'])**2))
    mae = np.mean(np.abs(combined_preds_df['ensemble_est'] - combined_preds_df['true_r']))
    
    results.append({'w_svd': w_svd, 'w_absa': w_absa, 'rmse': rmse, 'mae': mae})
    print(f"w_svd: {w_svd:.1f}, w_absa: {w_absa:.1f}, RMSE: {rmse:.4f}, MAE: {mae:.4f}")

    if rmse < best_rmse:
        best_rmse = rmse
        best_w_svd = w_svd
        best_w_absa = w_absa
    
    if mae < best_mae:
        best_mae = mae
        best_w_svd_mae = w_svd
        best_w_absa_mae = w_absa


print("\n--- 앙상블 결과 (15차원 ABSA 피처 반영) ---")
print(f"최적의 RMSE: {best_rmse:.4f}")
print(f"최적의 가중치 (SVD: {best_w_svd:.1f}, ABSA: {best_w_absa:.1f})")
print(f"최적의 MAE: {best_mae:.4f}")
print(f"최적의 가중치 (SVD: {best_w_svd_mae:.1f}, ABSA: {best_w_absa_mae:.1f})")

사용할 ABSA 피처 컬럼: ['sentiment_vector_0', 'sentiment_vector_1', 'sentiment_vector_2', 'sentiment_vector_3', 'sentiment_vector_4', 'sentiment_vector_5', 'sentiment_vector_6', 'sentiment_vector_7', 'sentiment_vector_8', 'sentiment_vector_9', 'sentiment_vector_10', 'sentiment_vector_11', 'sentiment_vector_12', 'sentiment_vector_13', 'sentiment_vector_14']
총 ABSA 피처 개수: 15개
총 고유 사용자 수: 27,807명
총 고유 비즈니스 수: 6,831개
총 집계된 평점/ABSA 데이터 수: 428,953개

--- ABSA 기반 아이템-아이템 유사도 행렬 계산 중 (시간 소요) ---
ABSA 기반 아이템-아이템 유사도 행렬 계산 완료.

--- 하이브리드 앙상블 모델 K-Fold 교차 검증 시작 (15차원 ABSA 피처 반영) ---

Fold 1/5 - 학습 및 예측 중...


Fold 1 ABSA Prediction: 100%|██████████| 85791/85791 [00:03<00:00, 22514.30it/s]



Fold 2/5 - 학습 및 예측 중...


Fold 2 ABSA Prediction: 100%|██████████| 85791/85791 [00:03<00:00, 23181.45it/s]



Fold 3/5 - 학습 및 예측 중...


...


Fold 4/5 - 학습 및 예측 중...


Fold 4 ABSA Prediction: 100%|██████████| 85790/85790 [00:03<00:00, 22893.59it/s]



Fold 5/5 - 학습 및 예측 중...


Fold 5 ABSA Prediction: 100%|██████████| 85790/85790 [00:03<00:00, 22844.66it/s]



--- 앙상블 가중치 조합 탐색 시작 (15차원 ABSA 피처 반영) ---
w_svd: 0.1, w_absa: 0.9, RMSE: 1.1091, MAE: 0.8576
w_svd: 0.2, w_absa: 0.8, RMSE: 1.0927, MAE: 0.8478
w_svd: 0.3, w_absa: 0.7, RMSE: 1.0789, MAE: 0.8397
w_svd: 0.4, w_absa: 0.6, RMSE: 1.0678, MAE: 0.8332
w_svd: 0.5, w_absa: 0.5, RMSE: 1.0596, MAE: 0.8285
w_svd: 0.6, w_absa: 0.4, RMSE: 1.0543, MAE: 0.8256
w_svd: 0.7, w_absa: 0.3, RMSE: 1.0520, MAE: 0.8246
w_svd: 0.8, w_absa: 0.2, RMSE: 1.0526, MAE: 0.8254
w_svd: 0.9, w_absa: 0.1, RMSE: 1.0562, MAE: 0.8280

--- 앙상블 결과 (15차원 ABSA 피처 반영) ---
최적의 RMSE: 1.0520
최적의 가중치 (SVD: 0.7, ABSA: 0.3)
최적의 MAE: 0.8246
최적의 가중치 (SVD: 0.7, ABSA: 0.3)
